In [1]:
import financials as fin
from imports import *

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Define the list of ticker symbols
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'BABA', 'PYPL', 'NVDA', 'SHOP',
          'COIN', 'TSLA', 'MU', 'BA', 'SIEB', 'INTEL' , 'PLTR', 'AMD', 'BAC', 
           'M', 'NFLX', 'V', 'PG', 'DIS', 'T', 'PFE', 'WMT', 'PEP', 'CRM', 'TMO', 
           'LLY', 'ORCL', 'MCD', 'WFC', 'C', 'AMGN', 'TSMC', 'ASML']

print(f"Fetching data for all tickers...")
financial_data = {}
for ticker in tickers:

    data = fin.get_financials(ticker)
    if data:
        financial_data[ticker] = data
    else:
        print(f"Failed to fetch data for {ticker}.")

financial_df = fin.extract_metrics(financial_data)

if 'Ticker' not in financial_df.columns:
    print("\n'Ticker' column is missing. Adding it manually.")
    financial_df['Ticker'] = tickers[:len(financial_df)]
    
numeric_cols = financial_df.select_dtypes(include=[np.number]).columns
financial_df[numeric_cols] = financial_df[numeric_cols].fillna(financial_df[numeric_cols].mean())
print('Fetched and filled data with nan')

Fetching data for all tickers...
Fetched and filled data with nan


In [2]:
print("\nScaling features...")
# Exclude 'Ticker' from features
features = financial_df.drop(['Ticker'], axis=1)

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the features
scaled_features = scaler.fit_transform(features)

# Create a scaled DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=features.columns)
scaled_df['Ticker'] = financial_df['Ticker'].values

# Define the Target Variable
print("\nDefining the target variable (Financial Health)...")
median_margin = financial_df['Net Margin'].median()
scaled_df['Financial_Health'] = (financial_df['Net Margin'] > median_margin).astype(int)

#print("\nTarget Variable:")
#print(scaled_df[['Ticker', 'Net Margin', 'Financial_Health']])

print("\nSplitting data into training and testing sets...")
X = scaled_df.drop(['Ticker', 'Financial_Health'], axis=1)
y = scaled_df['Financial_Health']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

# Train the Random Forest Classifier

print("\nTraining Random Forest Classifier...")
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
print("Random Forest model trained.")


Scaling features...

Defining the target variable (Financial Health)...

Splitting data into training and testing sets...
Training samples: 29, Testing samples: 8

Training Random Forest Classifier...
Random Forest model trained.


In [3]:
# Step 9: Predicting Financial Health of a Specific Company (XYZ)

xyz_ticker = 'NOK' 

print(f"\nPredicting financial health for {xyz_ticker}...")

# Predict financial health for the specified ticker
fin.predict_financial_health(xyz_ticker, scaler, rf, financial_data, financial_df, features)


Predicting financial health for NOK...

Random Forest Prediction for NOK: Unhealthy (Probability: 0.99)
